In [ ]:
import cv2
import os

def evaluate(upsampled, orig):
    psnr = cv2.PSNR(upsampled, orig)
    q, _ = cv2.quality.QualitySSIM_compute(upsampled, orig)
    ssim = (q[0] + q[1] + q[2]) / 3
    return round(psnr, 3), round(ssim, 3)

def upsample(img, alg_name, scale):
    # 创建SR对象
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    # 加载model
    path = "./model/LapSRN_x4.pb"
    sr.readModel(path)
    sr.setModel(alg_name, scale)
    # 下采样
    result = sr.upsample(img)
    return result

if __name__ == "__main__":
    # 输入图像
    img_path = "./set5/baby.png"
    # 放大系数
    scale = 4
    # model
    model = "./model/LapSRN_x4.pb"
    # 裁剪
    img = cv2.imread(img_path)
    width = img.shape[0] - (img.shape[0] % scale)
    height = img.shape[1] - (img.shape[1] % scale)
    cropped = img[0:width, 0:height]
    # Downscale the image for benchmarking
    # 缩小图像，以实现基准质量测试
    img_downscaled = cv2.resize(cropped, None, fx=1.0 / scale, fy=1.0 / scale)
    img_new = upsample(img_downscaled, scale)

    # 创建文件夹保存处理后的图像
    output_folder = f"./output"
    os.makedirs(output_folder, exist_ok=True)

    # 模型质量评估值
    psnr, ssim = evaluate(cropped, img_new)
    print("LapSRN超分辨率算法和不同差值的性能对比")
    print(f"PSNT:{psnr}, SSIM:{ssim}")

    # 保存超分辨率处理结果
    cv2.imwrite(f"{output_folder}/upscaled_image.png", img_new)
